In [1]:
import json

import math
import h5py
import numpy as np
import pandas
import cv2
from PIL import Image, ImageTk
from keras.layers import Dense, ZeroPadding2D, Convolution2D, MaxPooling2D, Dropout
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from scipy.ndimage import imread
from sklearn.utils import shuffle
from collections import defaultdict
import scipy
from scipy.misc import imresize
from scipy.misc import toimage
import cv2, numpy as np

import csv


Using TensorFlow backend.


In [26]:
global data, y_train
stats = defaultdict(float)

In [27]:
#from model_utils import read_shuffle_split_data, get_training_data_generator, get_validation_data_generator

In [28]:
scipy.__version__

'0.18.1'

In [29]:
data = pandas.read_csv(
        'D:/Self Driving Car Engineer/P3/data/driving_log.csv',
        names=['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed'])

In [30]:
line_number = np.random.randint(len(data))
data_to_process = data.iloc[[line_number]].reset_index()  

In [31]:
data.iloc[[line_number]].reset_index()  

,index,center,left,right,steering,throttle,brake,speed
0,4218,IMG/center_2016_12_01_13_39_45_674.jpg,IMG/left_2016_12_01_13_39_45_674.jpg,IMG/right_2016_12_01_13_39_45_674.jpg,0,0.783815,0,30.13503


In [32]:
#shift_ang = 55

In [11]:
#data_to_process['steering'][0]

In [33]:
# Turn RGB colour image to Hue, saturation and value and randomizes the value to change the brightness.
#
#In computer vision you often want to separate color components from intensity for various reasons, 
#such as robustness to lighting changes, or removing shadows. 
#
# 
def randomize_brightness(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    random_brightness = .1 + np.random.uniform()
    image[:,:,2] = image[:,:,2] * random_brightness
    image = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)
    return image

In [34]:
#crop the image so not to include too much noise and just learn on the road sections
img_width = 64
img_height = 64

batch_size = 256
epochs = 1

def random_generator():
    global data, stats
    
    data = shuffle(data)
    images = np.zeros((batch_size, img_width, img_height, 3))
    steerings = np.zeros(batch_size)    
    
    #randomly select an image, this will help in the learning process
    while True:
        for i_batch in range(batch_size):
            line_number = np.random.randint(len(data))
            data_to_process = data.iloc[[line_number]].reset_index()                    
            # Get center, left or right image, adjust steering accordingly
            rand = np.random.randint(3)
            if (rand == 0):
                path_file = data_to_process['left'][0].strip()
                shift_ang = .3
            elif (rand == 1):
                path_file = data_to_process['right'][0].strip()
                shift_ang = -.3
            else:
                path_file = data_to_process['center'][0].lstrip()
                shift_ang = 0
            
            if data_to_process['steering'][0].lstrip() == 'steering': #skips the data heading with 'steering'
                continue                  
            steering = float(data_to_process['steering'][0].lstrip()) + shift_ang
            
            #normalize image, add brightness, crop
            image = cv2.imread(path_file.strip())
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = randomize_brightness(image)
            shape = image.shape
            image = image[math.floor(shape[0] / 5.):shape[0] - 25, 0:shape[1]]
            image = cv2.resize(image, (img_height, img_width), interpolation=cv2.INTER_AREA)
            image = image / 255.
            image = np.array(image)

            #flip the image 50% of the time - thanks to my mentor for this - this will prevent the model from
            # bias and overfitting
            if np.random.randint(2) == 0:
                image = cv2.flip(image, 1)
                steering = -steering
                
            images[i_batch] = image
            steerings[i_batch] = steering
            stats[round(steering, 1)] += 1
        yield images, steerings

In [35]:
def load_data():
    global data, y_train

    data = pandas.read_csv('D:/Self Driving Car Engineer/P3/data/driving_log.csv', names=['center', 'left', 'right', 'steering', 'throttle', 'break', 'speed'])    
    y_train = data['steering'].values[1:]  #skip the header with 'steering'

In [36]:
y_train = data['steering'].values[1:]

In [37]:
len(y_train)

8036

In [41]:
def Network(input_shape):
    model = Sequential()
    
    #32 filters of size 3x3, number of samples is 256 x 32 = 8192
    model.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height, 3)))
    model.add(Convolution2D(32, 3, 3, activation='elu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(32, 3, 3, activation='elu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height, 3)))
    model.add(Convolution2D(64, 3, 3, activation='elu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='elu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))    
    model.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height, 3)))
    model.add(Convolution2D(128, 3, 3, activation='elu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='elu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))    
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(256, activation='elu'))
    model.add(Dropout(0.5))    
    model.add(Dense(64, activation='elu'))
    model.add(Dense(1))
    
    return model

In [42]:
if __name__ == '__main__':
    global y_train, stats

    load_data()

    input_shape = (3, img_width, img_height)
    model = Network(input_shape)
    opt = Adam(lr=0.0001)
    model.compile(loss='mse', optimizer=opt)
    
    for i in range(6):
        train_generator = random_generator()
        train_validator = random_generator()
        model.fit_generator(train_generator, samples_per_epoch=8192, nb_epoch=epochs, validation_data=train_validator, nb_val_samples=len(y_train) / 6)


    with open('model.json', 'w') as fd:
        json.dump(model.to_json(), fd)

    model.save_weights('C:/CarND-Transfer-Learning-Lab/model.h5')
 

Epoch 1/1
8192/8192 [==============================] - 233s - loss: 0.0732 - val_loss: 0.0672
Epoch 1/1
8192/8192 [==============================] - 231s - loss: 0.0548 - val_loss: 0.0488
Epoch 1/1
8192/8192 [==============================] - 230s - loss: 0.0452 - val_loss: 0.0486
Epoch 1/1
8192/8192 [==============================] - 231s - loss: 0.0402 - val_loss: 0.0393
Epoch 1/1
8192/8192 [==============================] - 232s - loss: 0.0393 - val_loss: 0.0415
Epoch 1/1
8192/8192 [==============================] - 233s - loss: 0.0388 - val_loss: 0.0439
